# Advent of Code 2023 - Day 2

In [137]:
import java.io.File

data class Game(
    val id: Int,
    val rounds: List<Round>,
)

class Round(map: Map<String, Int>) {
    private val defaultMap = map.withDefault { 0 }
    val red: Int by defaultMap
    val green: Int by defaultMap
    val blue: Int by defaultMap
}

val idRegex: Regex = """Game (\d+)""".toRegex()
val colorRegex: Regex = """(\d+) (\w+)""".toRegex()

val games: List<Game> = File("Day02.input.txt")
    .bufferedReader()
    .lineSequence()
    .map { line ->
        val (id, rounds) = line.split(":")
        Game(
            id = idRegex.find(id)!!.destructured.component1().toInt(),
            rounds = rounds.split(";").flatMap { roundLine ->
                roundLine.split(",")
                    .map { colorLine ->
                        colorRegex.findAll(colorLine)
                            .map { it.destructured }
                            .associate { (number, color) -> color to number.toInt() }
                    }
                    .map(::Round)
            }
        )
    }
    .toList()
    
data class Bag(val red: Int, val green: Int, val blue: Int)
val part1Bag = Bag(red = 12, green = 13, blue = 14)

## Part 1

Determine whether a given `Game` is impossible by checking if any of its `Round`s has more cubes (`red`, `green`, and `blue`) than what is in the `Bag`. Sum the `Game::id` of all possible games to get the answer.

In [138]:
fun Round.isPossible(bag: Bag): Boolean = red <= bag.red && green <= bag.green && blue <= bag.blue
fun Game.isPossible(bag: Bag): Boolean = rounds.all { it.isPossible(bag) }

games.filter { it.isPossible(part1Bag) }.sumOf(Game::id)


2237

### Notes

The most challenging section of this part was probably during input parsing. Once the structures are in place, determining which games are possible is pretty straight forward.

## Part 2

The power of a set of cubes is equal to the number of `red`, `green`, and `blue` cubes multiplied together. Determine each `Game`'s minimum number of cubes that would need to be in the bag to make each `Game` possible and sum their powers together.

In [139]:
import kotlin.reflect.KProperty1

fun Bag.power(): Int = red * green * blue

fun List<Round>.minOf(property: KProperty1<Round, Int>) = maxOf { property(it) }

fun Game.minBag(): Bag = Bag(
    red = rounds.minOf(Round::red),
    green = rounds.minOf(Round::green),
    blue = rounds.minOf(Round::blue),
)

games.map { it.minBag() }.sumOf { it.power() }

66681

### Notes

Again, I think this part was straight forward due to the setup in the input parsing. I think the relationship between a `Round` and a `Bag` could probably be better, maybe a parent interface `Cubes`, so that way we could protect against any refactors that might assign `Bag::red` to something other than `Round::red`.

Input parsing could definitely be cleaned up since, in hindsight, was the meat of the problem, so probably should have required more care. But, it's day 2, and I'm feeling a bit lazy.